1. Test daten abspeichern und hier einlesen
2. Ereigniskenndaten einlesen von csv
3. zu den listenobjekten jeweils eventdaten hinzufügen (Dauer, hN, imax_60, Jährlichkeit)
4. Benchmark eventsauswählen
5. load in_scaler und out_scaler
6. testdaten sequenzieren und normalisieren
7. daten vorhersagen
8. predictions mit delta time index versehen
9. Auswahl plotten mit delta time
    - wie plotten? evtl niederschlag mit balken von oben und predictions + actual ganglinie von unten
    - ereigniss kenndaten angeben (Dauer, hN, imax_60, Jährlichkeit)

Auswertung aller Sequencen
- MAE
- RMSE
- Scatterplott Actual vs Fitted
- Boxplot mit quantilen für jeden Vorhersagezeitschritt
- siehe Plots burrichter


